# Load and modify data

In [1]:
# Setup
import os

class Config:
    def __init__(self, for_git=True):
        self.for_git = for_git
        self.visualize = not for_git
        self.csv_separator = ',' if for_git else ';'
        self.csv_decimal = '.' if for_git else ','
        self.only_included_papers = True
        self.properties = ['source']
        self.proximity_mode = "sqrt"  # "log" mode is untested/unsafe, prefer "sqrt"
        self.base_path = 'data/'  # Default base path
        self.subset_path = 'data_subset/'
        self.visualization_path = 'visualization/'
        self.papers_path = 'G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR/02_nlp'
        self.review_path = 'G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR/03_notes'
        self.csv_file = 'C:/workspace/borgnetzwerk/tools/scripts/SLR/data.csv'
        self.gap_too_large_threshold = 1000
        self.savetime_on_fulltext = False   # If True, operations on fulltext will be kept to a minimum
        self.try_to_save_time = False
        self.recalculate_pos_in_paper = False # while the calculation is inefficient, just load from file
        self.debug = True


    def get_output_path(self, path="", visualization=False):
        """
        Determine the output path based on the configuration and parameters.
        """
        if not path:
            path = self.subset_path if self.only_included_papers else self.base_path
        if visualization and not path.endswith(self.visualization_path):
            path = os.path.join(path, self.visualization_path)
        if not os.path.exists(path):
            os.makedirs(path)
        return path

# Usage
config = Config(for_git=True)


In [2]:
# debug test
import numpy as np
from typing import List, Tuple

def run_debug_test(config: Config, instances:list[str] = None, papers:list[str] = None, paper_instance_occurrence_matrix:np.ndarray = None):
    if "ikewiki" in instances:
        ike_index = instances.index("ikewiki")
        sum_ikewiki = np.sum(paper_instance_occurrence_matrix[:,ike_index])
        if sum_ikewiki > 1:
            raise Exception(f"Only one paper should contain 'ikewiki', but {sum_ikewiki} do.")


In [3]:
import time

def time_function(func):
    def wrapper(*args, **kwargs):
        appendix = ""
        # if instances in args:
        if "instances" in kwargs:
            # append len of instances
            appendix = f"({len(kwargs['instances'])} instances"
        if "papers" in kwargs:
            if appendix:
                appendix += ", "
            appendix += f"{len(kwargs['papers'])} papers"
        if appendix:
            appendix += ")"
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} executed in {end_time - start_time} seconds" + appendix)
        return result
    return wrapper

In [4]:
# helpful functions
def index_all(input_list, item):
    return [i for i, x in enumerate(input_list) if x == item]

In [5]:
# Extract Paper Metadata
from bnw_tools.extract import util_zotero


def get_paper_nlp_paths(papers_path):
    paper_nlp_paths = {}
    for file in os.listdir(papers_path):
        if file.endswith(".json"):
            paper_nlp_paths[file[:-5]] = os.path.join(papers_path, file)
    return paper_nlp_paths

def reduce_to_reviewed_papers(papers, review_path):
    # todo: sort by review score + average rank
    included_papers = []
    excluded_papers = []
    included_identifier = ["review_score:: 3", "review_score:: 4", "review_score:: 5"]
    excluded_identifier = ["review_score:: 2", "review_score:: 1", "review_score:: 0"]
    for file in os.listdir(review_path):
        if file.endswith(".md"):
            paper_name = file[:-3]
            if paper_name in papers:
                # check if file contains "reviewed"ArithmeticError
                with open(os.path.join(review_path, file), 'r', encoding="utf8") as f:
                    content = f.read()
                    for id in included_identifier:
                        if id in content:
                            included_papers.append(paper_name)
                            break
                    for id in excluded_identifier:
                        if id in content:
                            excluded_papers.append(paper_name)
                            break
    return included_papers, excluded_papers

@time_function
def get_paper_metadata(papers, path):
    papers_metadata = {}

    bib_resources = util_zotero.BibResources(path)

    for paper in papers:
        for entry in bib_resources.entries:
            if hasattr(bib_resources.entries[entry], 'file') and paper in bib_resources.entries[entry].file:
                papers_metadata[paper] = bib_resources.entries[entry].get_dict()
                del bib_resources.entries[entry]
                break


    print(f"{len(papers_metadata)} out of {len(papers)} papers have metadata.")

    return papers_metadata

paper_nlp_paths = get_paper_nlp_paths(config.papers_path)

papers = list(paper_nlp_paths.keys())


In [6]:
def exclude_papers(paper_nlp_paths, papers, included_papers, only_included_papers):
    if only_included_papers:
        deletions = [False] * len(papers)
        for p_ID, paper in enumerate(paper_nlp_paths):
            if paper not in included_papers:
                deletions[p_ID] = True
        for p_ID, deletion in enumerate(deletions):
            if deletion:
                paper_nlp_paths.pop(papers[p_ID])
        papers = included_papers
        print("Only included papers are considered. Excluded or not reviewed papers are removed.")
    return paper_nlp_paths, papers

included_papers, excluded_papers = reduce_to_reviewed_papers(papers, config.review_path)

paper_nlp_paths, papers = exclude_papers(paper_nlp_paths, papers, included_papers, only_included_papers = config.only_included_papers)

# Free memory
del included_papers

Only included papers are considered. Excluded or not reviewed papers are removed.


In [7]:
# extract the matadata from bibtex
papers_metadata = get_paper_metadata(papers, 'G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR')

Found no new Zotero export at G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR:
There should be a folder called 'files'
We now have 1035 PDFs stored at G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR\00_PDFs
164 out of 164 papers have metadata.
get_paper_metadata executed in 15.394164800643921 seconds


In [8]:
# # sort papers by year
papers = sorted(papers, key=lambda x: papers_metadata[x]['year'] if x in papers_metadata and 'year' in papers_metadata[x] else "9999")
# sort paper_nlp_paths and papers_metadata accordingly
paper_nlp_paths = {k: paper_nlp_paths[k] for k in papers}
papers_metadata = {k: papers_metadata[k] for k in papers if k in papers_metadata}

In [9]:
# Step 1: find occurrences of instances in bag of words of papers
import pandas as pd
import os
import json
import numpy as np


def split_string(input_string, delimiters = [" ", "-", "_"]):
    for delimiter in delimiters:
        input_string = " ".join(input_string.split(delimiter))
    return input_string.split()

def csv_to_dict_of_sets(csv_file, config: Config, prune_nan = True):
    dict_of_sets = {}
    # try:
    #     df = pd.read_csv(csv_file)
    # except pd.errors.ParserError:
    #     print("Error parsing CSV file. Trying again with 'error_bad_lines=False'")
    # TODO: Specify modular separator and decimal here as well 
    try:
        df = pd.read_csv(csv_file, on_bad_lines='warn', delimiter=config.csv_separator,  encoding="utf-8")
    except:
        print("Error parsing CSV file. Trying again with 'encoding=ISO-8859-1'")
        df = pd.read_csv(csv_file, on_bad_lines='warn', delimiter=config.csv_separator, encoding='ISO-8859-1')
    for column in df.columns:
        dict_of_sets[column] = set(df[column].str.lower())
        if prune_nan and np.nan in dict_of_sets[column]:
            dict_of_sets[column].remove(np.nan)
    # saved_column = df['process'] #you can also use df['column_name']
    # delete all that exists in two or more columns
    for key in dict_of_sets:
        for other_key in dict_of_sets:
            if key != other_key:
                dict_of_sets[key] = dict_of_sets[key].difference(dict_of_sets[other_key])
    return dict_of_sets

def prune_properties(instance_types_dicts, properties_to_prune = [], prune_empty = True, prune_x = True):
    properties = {}

    
    # merge "interchange format" into "data format specification"
    if 'interchange format' in instance_types_dicts:
        pruned = []
        for key in instance_types_dicts['interchange format']:
            if len(key) > 1:
                instance_types_dicts['data format specification'].add(key)
                pruned.append(key)
        for key in pruned:
            instance_types_dicts['interchange format'].remove(key)

    for instance_type in instance_types_dicts:
        prune = False
        if instance_type in properties_to_prune:
            prune = True
        elif prune_empty and len(instance_types_dicts[instance_type]) == 0:
            # prune empty sets
            prune = True
        elif prune_x and len(max(instance_types_dicts[instance_type], key=len)) < 2:
            # prune sets with only one character entries
            prune = True
        
        if prune:
            properties[instance_type] = instance_types_dicts[instance_type]

    for instance_type in properties:
        instance_types_dicts.pop(instance_type)           

    return instance_types_dicts, properties

def count_occurrences(papers, instances):
    occurrences = np.zeros((len(papers), len(instances)), dtype=int)

    for p, paperpath in enumerate(papers.values()):
        with open(paperpath, 'r', encoding="utf8") as f:
            paper = json.load(f)
            for i, instance in enumerate(instances):
                present = True
                pieces = split_string(instance)
                for piece in pieces:
                    if piece.lower() not in paper['bag_of_words']:
                        present = False
                        break
                    
                # if instance == "system integration":
                #     if "Liu und Hu - 2013 - A reuse oriented representation model for capturin" in paperpath:
                #         print(present)
                if present:
                    occurrences[p][i] = 1
    return occurrences

# ---------------------- Variables ----------------------

## instances: A list of all instances, regardless of their type
# first all type 1, then all type 2, etc.
# if possible, instance sare ordered by their occurrence

## instances_dicts: A dictionary of all different types (columns) of instances
#
# types:
#  - process
#  - software
#  - data item
#  - data model
#  - data format specification
#  - interchange format
#  - source
#
# instances_dicts['process']: A set of all instances of the type 'process'
#
instance_types_dicts = {}

## paper_nlp_dict: A dictionary of all papers and their NLP data (as dict)

## occurrences: A matrix of binary occurrences of instances in papers
#
# rows: papers
# columns: instances
# cells: 1 if instance is present in paper, 0 otherwise
#
paper_instance_occurrence_matrix = np.zeros((), dtype=int)


# ---------------------- Main ----------------------

# Usage example

instance_types_dicts = csv_to_dict_of_sets(config.csv_file, config)

# Extract instance types that are actually property types
instance_types_dicts, property_types_dicts = prune_properties(instance_types_dicts, properties_to_prune=config.properties)

def get_instances_list(instance_types_dicts):
    instances = []
    # merge all sets into one set
    for instance_type in instance_types_dicts:
        instances += (instance_types_dicts[instance_type])
    return instances

instances = get_instances_list(instance_types_dicts)

paper_instance_occurrence_matrix = count_occurrences(paper_nlp_paths, instances)

# free unneeded memory
del paper_nlp_paths

In [10]:
# debug
run_debug_test(config, instances, papers, paper_instance_occurrence_matrix)

In [11]:
def sort_instances(config: Config,matrix, instances, instance_types_dict=None,):
    # total occurrences of each instance
    instance_occurrences = {}
    
    for i, instance in enumerate(instances):
        instance_occurrences[instance] = matrix[:, i].sum()
    sorted_instances = {k: float(v) for k, v in sorted(instance_occurrences.items(), key=lambda item: item[1], reverse=True) if v > 0}
    filepath = os.path.join(config.get_output_path(),'instance_occurrences')
    with open(filepath + '.json', 'w', encoding="utf-8") as f:
        json.dump(sorted_instances, f, ensure_ascii=False, indent=4)

    sorted_instance_list = list(sorted_instances.keys())


    if instance_types_dict is not None:
        # Instances should be sorted by their type
        type_lists = [[] for _ in range(len(instance_types_dict))]
        for instance in sorted_instance_list:
            for type_ID, instance_type in enumerate(instance_types_dict):
                if instance in instance_types_dict[instance_type]:
                    type_lists[type_ID].append(instance)
        type_sorted_instances = [item for sublist in type_lists for item in sublist]

    new_order = [0] * len(sorted_instance_list)
    for i, instance in enumerate(type_sorted_instances):
        new_order[i] = instances.index(instance)

    return type_sorted_instances, new_order

def remove_zeros(matrix, columns=True, rows=True, row_lists=None, column_lists=None):
    # remove all columns that are all zeros
    if columns:
        deleted_columns = np.all(matrix == 0, axis=0)
        matrix = matrix[:, ~np.all(matrix == 0, axis=0)]

    # remove all rows that are all zeros
    if rows:
        deleted_rows = np.all(matrix == 0, axis=1)
        matrix = matrix[~np.all(matrix == 0, axis=1)]

    
    return matrix, [deleted_columns, deleted_rows]

def update_instances(matrix, instances, instance_types_dict=None):
    instances, new_order = sort_instances(config, matrix, instances, instance_types_dict)

    new_order = np.array(new_order)
    matrix = matrix[:, new_order]
    
    matrix, deletions = remove_zeros(matrix)
    return matrix, instances, deletions

paper_instance_occurrence_matrix, instances, deletions = update_instances(paper_instance_occurrence_matrix, instances, instance_types_dicts)

def handle_deletions(input, deletions, rows = True):
    """
    input: list, dict or np.ndarray
    deletions: list of bools
    rows: if True, deletions[1] is used, else deletions[0]
    """
    delID = 1 if rows else 0

    if deletions[delID].any():
        # rows were deleted, in this case: papers
        if isinstance(input, list):
            input = [item for i, item in enumerate(input) if not deletions[delID][i]]
        elif isinstance(input, dict):
            input = {key: item for i, (key, item) in enumerate(input.items()) if not deletions[delID][i]}
        elif isinstance(input, np.ndarray):
            input = input[~deletions[delID]]
    return input

papers = handle_deletions(papers, deletions)
# free unneeded memory
del deletions

In [12]:
run_debug_test(config, instances, papers, paper_instance_occurrence_matrix)

In [13]:
# get all text files
def get_paper_full_text(directory):
    paper_full_text = {}
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".txt"):
                    file_path = os.path.join(folder_path, file)
                    paper_full_text[file[:-4]] = file_path
                    break

    return paper_full_text

paper_full_text = get_paper_full_text('G:/Meine Ablage/SE2A-B42-Aerospace-knowledge-SWARM-SLR/00_PDFs')


In [14]:
# Step 2: find occurrences of instances in full text of papers
import sys
from bisect import bisect_left
from sortedcontainers import SortedSet


# Step 2: find occurrences of instances in full text of papers
import sys
from bisect import bisect_left
from sortedcontainers import SortedSet
import numpy as np
import json

class PosInPaper:
    def __init__(self):
        # List of paper identifiers
        self.papers = []
        # List of literals
        self.literals = []
        # Dict of unique words across all literals
        self.words = {}
        self.word_len = []
        # List of unique combinations of words across all literals
        self.word_combinations = {}
        self.word_combination_lists = []
        self.word_combination_index_literal = {}
        # 2D list mapping pairs of literals to their word combination index
        self.word_combination_index_literal_literal = []
        # 2D list of SortedSets, each containing the positions of a word in a paper
        self.word_occurrences_in_papers = []
        # 3D list containing the minimum distances between word combinations in each paper
        self.min_distances = []

    @time_function
    def populate(self, config: Config, papers: list, literals: list[str], paper_full_text, optimize=True):
        """
        Populates the internal data structures with occurrences and distances of literals in papers.

        Parameters:
        - config (Config): Configuration object containing settings.
        - papers (list): List of paper identifiers.
        - literals (list[str]): List of literals to process.
        - paper_full_text (dict): Mapping from paper identifiers to their full text file paths.
        - optimize (bool): Flag to optimize data structures after population.
        """
        self.initialize_variables(papers, literals)
        self.process_literals(literals)
        self.process_literal_combinations(literals)
        self.setup_data_structures(papers)
        self.find_occurrences_in_texts(papers, paper_full_text)
        if optimize:
            self.optimize_data()

    def update_list_attribute(self, list, name):
        existing = getattr(self, name)
        if not existing:
            setattr(self, name, list)
        else:
            if existing != list:
                print(f"Warning: {name} has changed.")
                for item in list:
                    if item not in existing:
                        print(f"Item {item} is new.")
                        existing.append(item)
                # update
                raise NotImplementedError("create a function to update other relying attributes")


    @time_function
    def initialize_variables(self, papers, literals):
        """
        Initializes basic variables for the class instance.

        Parameters:
        - papers (list): List of paper identifiers.
        - literals (list): List of literals to process.
        """
        if not self.papers:
            self.papers = papers
        else:
            if self.papers != papers:
                print("Warning: Papers have changed.")
                for paper in papers:
                    if paper not in self.papers:
                        print(f"Paper {paper} is new.")
                        self.papers.append(paper)
        self.papers = papers
        self.literals = literals
        self.word_combination_index_literal = {lit: None for lit in literals}

    @time_function
    def process_literals(self, literals):
        """
        Processes each literal to extract and store unique words and word combinations.

        Parameters:
        - literals (list): List of literals to process.
        """
        for lit in literals:
            word_list = split_string(lit)
            self.add_words(word_list)
            self.add_if_word_combination(word_list, lit)

    def add_words(self, word_list):
        """
        Adds unique words from a list to the internal list of words.

        Parameters:
        - word_list (list): List of words to add.
        """
        for word in word_list:
            if word not in self.words:
                self.words[word] = len(self.words)
                self.word_len.append(len(word))

    def add_if_word_combination(self, word_list, lit):
        """
        Adds a unique combination of words from a list to the internal list of word combinations.

        Parameters:
        - word_list (list): List of words forming a combination.
        - lit (str): The literal corresponding to the word combination.
        """
        if len(word_list) > 1:
            pos = self.word_combination_index_literal.get(lit, -1)
            if pos == -1 or pos == None:
                froz = frozenset(word_list)
                pos = len(self.word_combinations)
                self.add_word_combination(froz, pos)
            self.word_combination_index_literal[lit] = pos
    
    def add_word_combination(self, froz, pos):
        self.word_combinations[froz] = pos
        self.word_combination_lists.append([self.words[word] for word in sorted(froz, key=len, reverse=True)])

    @time_function
    def process_literal_combinations(self, literals):
        """
        Processes combinations of literals to store their indices in the internal data structure.

        Parameters:
        - literals (list): List of literals to process.
        """
        self.word_combination_index_literal_literal = [[None] * len(literals) for _ in range(len(literals))]
        # Use a dictionary for quick lookup and storage
        combination_index = len(self.word_combinations)

        for id1, literal1 in enumerate(literals):
            for id2 in range(id1 + 1, len(literals)):
                literal2 = literals[id2]
                # Use a sorted tuple for consistent ordering
                froz = frozenset(split_string(literal1) + split_string(literal2))
                # Check if the combination is already in the dictionary
                pos = self.word_combinations.get(froz, -1)
                if pos == -1:
                    pos = combination_index
                    combination_index += 1

                    self.add_word_combination(froz, pos)

                # Update the matrix with the index of the combination
                self.word_combination_index_literal_literal[id1][id2] = pos
                self.word_combination_index_literal_literal[id2][id1] = pos

    @time_function
    def setup_data_structures(self, papers):
        """
        Initializes the data structures for storing word occurrences and minimum distances.

        Parameters:
        - papers (list): List of paper identifiers.
        """
        if self.word_occurrences_in_papers == []:
            self.word_occurrences_in_papers = [[SortedSet() for _ in self.words] for _ in papers]
        if isinstance(self.min_distances, list) and self.min_distances == [] or self.min_distances is None:
            self.min_distances = np.full((len(papers), len(self.word_combinations)), -2, dtype=int)

    @time_function
    def find_occurrences_in_texts(self, papers, paper_full_text):
        """
        Finds and stores the occurrences of each word in the full text of each paper.

        Parameters:
        - papers (list): List of paper identifiers.
        - paper_full_text (dict): Mapping from paper identifiers to their full text file paths.
        """
        for paperID, paper in enumerate(papers):
            if paper in paper_full_text:
                with open(paper_full_text[paper], 'r', encoding="utf8") as f:
                    text = f.read().lower()
                    for wordID, word in enumerate(self.words):
                        self.find_and_add_word_occurrences(paperID, wordID, word, text)
            else:
                print(f"Paper {paper} has no full text available.")

    def find_and_add_word_occurrences(self, paperID, wordID, word, text):
        """
        Finds and adds the occurrences of a word in a paper's text to the internal data structure.

        Parameters:
        - paperID (int): The index of the paper in the internal list.
        - wordID (int): The index of the word in the internal list.
        - word (str): The word to find occurrences of.
        - text (str): The full text of the paper.
        """
        pos = text.find(word)
        while pos != -1:
            self.word_occurrences_in_papers[paperID][wordID].add(pos)
            pos = text.find(word, pos + 1)

    @time_function
    def optimize_data(self):
        """
        Optimizes the internal data structures for faster access and smaller memory footprint.
        """
        # self.word_combination_index_literal_literal = np.array(self.word_combination_index_literal_literal, dtype=int)
        for paperID in range(len(self.papers)):
            for wordID in range(len(self.words)):
                # self.word_occurrences_in_papers[paperID][wordID] = SortedSet(self.word_occurrences_in_papers[paperID][wordID])
                self.word_occurrences_in_papers[paperID][wordID] = [(x, wordID) for x in self.word_occurrences_in_papers[paperID][wordID]]

    def save_to_csv(self, config:Config = None, path = None, name = "pos_in_paper"):
        if path is None:
            path = config.get_output_path()

        # save min_distances to csv
        # dump self.min_distances to csv, with self.papers as row headers and self.word_combinations as column headers
        filepath = os.path.join(path, name + '_min_distances.csv')
        with open(filepath, 'w', encoding='utf-8') as f:
            word_combinations = ["_".join(sorted(froz, key=len, reverse=True)) for froz in self.word_combinations.keys()]
            f.write("papers" + config.csv_separator + config.csv_separator.join(word_combinations) + '\n')
            for i, paper in enumerate(self.papers):
                f.write(paper + config.csv_separator + config.csv_separator.join(map(str, self.min_distances[i])) + '\n')
        


    @time_function
    def save_to_file(self, config, path=None, name = "pos_in_paper", check_size=False, min_distance_to_csv=False, backup = True):
        """
        Saves the internal data structures to files for persistence.

        Parameters:
        - path (str, optional): The base path for the output files. Defaults to "pos_in_paper".
        """
        if path is None:
            path = config.get_output_path()
        filepath = os.path.join(path, name + '.json')
        
        data = {}

        for key, value in self.__dict__.items():
            if key == "min_distances":
                data[key] = value.tolist()
            elif key == "word_combinations":
                data[key] = {"_".join(key): value for key, value in self.word_combinations.items()}
            else:
                data[key] = value
            if check_size:
                # Construct the file name for each sub-dictionary
                filepath = os.path.join(path, f"{name}_{key}.json")
                with open(filepath, 'w', encoding='utf-8') as f:
                    json.dump(data[key], f, ensure_ascii=False)
            pass

        if min_distance_to_csv:
            self.save_to_csv(config, path, name)
        
        if backup:
            backup_path = os.path.join(path, name + "_backup" + '.json')
            if os.path.exists(filepath):
                existing_is_healthy = True
                try:
                    self.load_from_file(config, path, name)
                except Exception as e:
                    print(f"Overwriting existing save")
                    existing_is_healthy = False
                if existing_is_healthy:
                    if os.path.exists(backup_path):
                        os.remove(backup_path)
                    os.rename(filepath, backup_path)
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)



    @time_function
    def load_from_file(self, config, path=None, name="pos_in_paper", backup=True):
        """
        Loads the internal data structures from files.

        Parameters:
        - path (str, optional): The base path for the input files. Defaults to "pos_in_paper".
        """
        if path is None:
            path = config.get_output_path()
        filepath = os.path.join(path, name + '.json')
        
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except Exception as e:
            print(f"Error loading file {filepath}: {e}")
            if backup:
                backup_path = os.path.join(path, name + "_backup" + '.json')
                if os.path.exists(backup_path):
                    print(f"Trying to load backup file {backup_path}")
                    with open(backup_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                else:
                    raise Exception(f"No backup file found at {backup_path}")
            else:
                raise e


        for key, value in data.items():
            if key == "word_combinations":
                self.word_combinations = {frozenset(split_string(sub_key)): i for i, sub_key in enumerate(value)}
            elif key == "min_distances":
                self.min_distances = np.array(data["min_distances"])
            else:
                try:
                    setattr(self, key, value)
                except Exception as e:
                    raise Exception(f"Error loading pos_in_paper attribute {key}: {e}")

        self.setup_data_structures(self.papers)
        
    @time_function
    def calculate_all_possible(self):
        """
        Calculates the minimum distances between all possible combinations of literals in all papers.
        """
        save_every = None
        if len(self.papers) > 300:
            print("Warning: This operation is computationally expensive and may take a long time.")
            save_every = 300
        for p in range(len(self.papers)):
            if save_every and p % save_every == 0:
                print(f"Processing paper {p} of {len(self.papers)}")
                self.save_to_file(config)
            for w in range(len(self.word_combinations)):
                self.find_min_distance_by_id(p, w)
            # for i in range(len(self.literals)):
            #     for j in range(i + 1, len(self.literals)):
            #         # get word_combination_index_literal_literal
                    # self.find_min_distance_by_id(p, self.word_combination_index_literal_literal[i][j])

    def find_min_distance_by_id(self, paperID, wcID, allow_call=True):
        """
        Finds the minimum distance between occurrences of literals in a paper.

        Parameters:
        - paper (str): The identifier for the paper.
        - literals (list): A list of literals for which the distance is to be found.
        - allow_call (bool): Flag to allow recursive call to get_min_distance.

        Returns:
        - int: The minimum distance between occurrences of the literals.
        """
        distance = self.min_distances[paperID][wcID]

        if distance == -1:
            # word combination not found in paper
            return -1
        if distance == -2:
            # calculate distance
            pass
        else:
            return distance
        
        list_ids = self.word_combination_lists[wcID]
        # since we have attached global Word IDs to the occurences, we need to map to their local position
        list_ids_map = {list_ids[i]: i for i in range(len(list_ids))}
        # literals = [list(self.words)[i] for i in list_ids]
        
        #TODO: It should be possible to remove smaller words from the list,
        # if a larger word contains it:
        # e.g. remove "engine" if "engineer" is in the list
        ## The following implementation works, but is not used for now. Reasons:
        ## 1. It could be slower than the current implementation
        ## 2. It might be beneficial to future use-cases to not remove smaller words
        # literals = [list(self.words)[key] for key in list_ids_map]
        # # check if any literal is a substring of another
        # for i, lit1 in enumerate(literals):
        #     for j, lit2 in enumerate(literals):
        #         if i != j and lit1 in lit2:
        #             # if lit1 is a substring of lit2, remove it from the list
        #             list_ids_map.pop(list_ids[i])
        #             break
        
        lit_len = [self.word_len[i] for i in list_ids]

        for i in list_ids:
            if not self.word_occurrences_in_papers[paperID][i]:
                self.min_distances[paperID][wcID] = -1
                return -1
        # Outsourced to optimize
        # inputs = [[(x, i) for x in self.word_occurrences_in_papers[paperID][wordID]] for i, wordID in enumerate(list_ids)]
        inputs = [self.word_occurrences_in_papers[paperID][wordID] for wordID in list_ids]

        indices = [lst[0][0] for lst in inputs]
        best = float('inf')

        for item in sorted(sum(inputs, [])):
            if item[0] not in indices:
                continue
            indices[list_ids_map[item[1]]] = item[0]
            arr_min = min(indices)
            best = min(max(indices) - arr_min - lit_len[indices.index(arr_min)], best)
            if best <= 0:
                best = 0
                break
        self.min_distances[paperID][wcID] = best

        return best

In [15]:
pos_in_paper = PosInPaper()

# config.recalculate_pos_in_paper = True # May be used for debug purposes

if not config.recalculate_pos_in_paper:
    try:
        pos_in_paper.load_from_file(config)
    # print exception
    except Exception as e:
        # if config.debug:
        #     raise e
        print(e)
        print("Starting from scratch.")
        
        config.recalculate_pos_in_paper = True

# TODO:
# raise NotImplementedError("Implement a check that compares the loaded instances and papers with the current ones")

if config.recalculate_pos_in_paper:
    pos_in_paper.populate(config, papers, instances, paper_full_text)
    # pos_in_paper.save_to_file(config)

setup_data_structures executed in 0.0 seconds
load_from_file executed in 2.3355462551116943 seconds


In [16]:
if config.recalculate_pos_in_paper or -2 in pos_in_paper.min_distances:
    pos_in_paper.calculate_all_possible()
    # Info: This method is extremely slow. requires more testing, which is currently done in a side project:
    ## scripts\SLR\MVP\test_case.ipynb
    pos_in_paper.save_to_file(config)
    config.recalculate_pos_in_paper = False

In [17]:
# Step 3: find the gap between the pieces of an instance
import sys

@time_function
def find_instance_piece_gap(config:Config, papers, paper_full_text, instances, paper_instance_occurrence_matrix, pos_in_paper:PosInPaper):
    error_matrix = np.zeros(paper_instance_occurrence_matrix.shape, dtype=float)
    for paperID, paper in enumerate(papers):
        if paperID % 100 == 0:
            # print(f"Processing paper {paperID} of {len(papers)}")
            pass
        if paper in paper_full_text:
            for i, instance in enumerate(instances):
                if paper_instance_occurrence_matrix[paperID][i] == 0:
                    continue
                wcID = pos_in_paper.word_combination_index_literal[instance]
                # TODO: handle if that instance has no word combination index entry
                if wcID is None:
                    # word has no distance
                    continue
                min_distance = pos_in_paper.find_min_distance_by_id(paperID, wcID)
                if min_distance is None:
                    pass
                if min_distance > config.gap_too_large_threshold:
                    # print(f"Gap for {instance} in {paper} ({min_distance} > {GAP_TOO_LARGE_THRESHOLD})")
                    paper_instance_occurrence_matrix[paperID][i] = 0
                    # get log base 10 of min distance
                    error_matrix[paperID][i] = round(np.log10(min_distance), 1)
                
                # Some pieces may not be found in the full text
                if min_distance == -1:
                    # print(f"{instance} not found in {paper} at all")
                    paper_instance_occurrence_matrix[paperID][i] = 0
                    error_matrix[paperID][i] = min_distance
                    # for these, we do not store the gap                    
                    continue

    return error_matrix

In [18]:
error_matrix = find_instance_piece_gap(config, papers, paper_full_text, instances, paper_instance_occurrence_matrix, pos_in_paper)

error_matrix, has_error = remove_zeros(error_matrix)
error_papers = handle_deletions(papers, has_error)
error_instances = handle_deletions(instances, has_error, rows = False)

paper_instance_occurrence_matrix, instances, deletions = update_instances(paper_instance_occurrence_matrix, instances, instance_types_dicts)

papers = handle_deletions(papers, deletions)

instance_instance_co_occurrence_matrix = np.dot(paper_instance_occurrence_matrix.T, paper_instance_occurrence_matrix)

# free unneeded memory
del deletions, has_error

find_instance_piece_gap executed in 0.040008544921875 seconds


In [19]:
# Create year_paper_occurrence_matrix
def create_year_paper_occurrence_matrix(papers_metadata, paper_instance_occurrence_matrix, papers, is_error_matrix=False):
    year_papers = {}

    for paper in papers_metadata:
        if 'year' in papers_metadata[paper]:
            year = int(papers_metadata[paper]['year'])
            if year not in year_papers:
                year_papers[year] = []
            year_papers[year].append(paper)


    earliest = min(year_papers)
    latest = max(year_papers)
    span = latest-earliest+1

    for year in range(earliest, latest):
        if year not in year_papers:
            year_papers[year] = []

    year_papers = {k: v for k, v in sorted(year_papers.items(), key=lambda item: item[0])}

    if is_error_matrix:
        # convert any value != 0 to 1
        paper_instance_occurrence_matrix = np.where(paper_instance_occurrence_matrix != 0, 1, 0)

    # create a year_instance_occurence matrix from the paper_instance_occurrence_matrix
    year_instance_occurrence_matrix = np.zeros((span, paper_instance_occurrence_matrix.shape[1]), dtype=int)
    for yearID, year in enumerate(year_papers):
        for paper in year_papers[year]:
            if paper in papers:
                paperID = papers.index(paper)
                year_instance_occurrence_matrix[yearID] += paper_instance_occurrence_matrix[paperID]
    
    return year_instance_occurrence_matrix, year_papers

year_instance_occurrence_matrix, year_papers = create_year_paper_occurrence_matrix(papers_metadata, paper_instance_occurrence_matrix, papers)

# Setup Complete

We now have:

| Variable                          | Type    | Size         | Comments |
|-----------------------------------|---------|--------------|----------|
| error_instances                   | list    | 165          | Comments |
| error_matrix                      | ndarray | (999, 165)   | Comments |
| error_papers                      | list    | 999          | Comments |
| gap_too_large_threshold           | int     | n.a.         | Comments |
| instance_piece_gap                | dict    | 151          | Comments |
| instance_types_dicts              | dict    | 5            | Comments |
| instances                         | list    | 315          | Comments |
| paper_full_text                   | dict    | 1029         | Comments |
| paper_instance_occurrence_matrix  | ndarray | (1003, 315)  | Comments |
| papers                            | list    | 1003         | Comments |
| pos_in_paper                      | dict    | 1003         | Comments |

Consisting of:
* The paper_instance_occurrence_matrix, binary listing if a term (instance) is present in a paper
  * papers x instances
* The error_matrix, of all instances that were dropped from the paper_instance_occurrence_matrix
  * error_papers x error_instances

And some leftover variables:
* instance_types_dicts, listing all instance types ("process", "software", ...) and their respective instance sets ("Curation", "Knowledge Work", ...)
* paper_full_text, containing each papers full text
  * pos_in_paper, listing for each paper: for each instance: each position of that instance in that papers full text.
* instance_piece_gap, a dict listing all instances made up from compound words (e.g. "Knowledge Work", and their minimum distance in each papers full text)
  * gap_too_large_threshold, defining how far appart a finding of "Knowledge" and "Work" would qualify as "Knowledge Work"

In [20]:
# ~3 min | {( len(papers) * len(instances) ) / (3 * 1000) }seconds  compare proximity of all instances with one antoher
# ~8 min right now.
# 3 min 30 sec with 164 papers and 339 instances
@time_function
def calculate_proximity_matrix(config:Config, pos_in_paper:PosInPaper, instances, mode = "sqrt", try_to_save_time = False):
    # TODO: Optimize this function.
    # each instance needs to have it's occurences as pieces clustered together, so that only those below max distance are considered

    # create a np zeros matrix of size instances x instances
    instance_instance_proximity_matrix = np.zeros((len(instances), len(instances)), dtype=float)
    
    # alternatives are:
    # "sqrt" - 1 / (square root of the distance)
    # "linear" - 1 / distance
    # "binary" - 1 if distance < MAX_GAP_THRESHOLD, 0 otherwise
    # "log" - 1 / log(distance) 
    
    # There is a chance that pos_in_paper papers and instances are out of sync with the current papers and instances
    paperIDs = [paperID for paperID, name in enumerate(pos_in_paper.papers) if name in papers] 
    lID_map = {instances.index(name):instanceID for instanceID, name in enumerate(pos_in_paper.literals) if name in instances}

    for id1 in range(len(instances)):
        # print (f"Processing {id1} of {len(instances)}: {instance1}")
        for id2 in range(id1+1, len(instances)):
            # FIXME: this resulted in a matrix which was not symmetric.
            # That hints at a problem with the calclulation, [id1][id2] and [id2][id1] should be the same
            wcID = pos_in_paper.word_combination_index_literal_literal[lID_map[id1]][lID_map[id2]]
            for paperID in paperIDs:
                distance = pos_in_paper.find_min_distance_by_id(paperID, wcID)
                
                if distance < 0:
                    # print(f"Error: {instance1} and {instance2} not found in {paper}")
                    continue
                result = 0.0
                if distance == 0:
                    result = 1
                elif distance == 1:
                    result = 1
                elif mode == "sqrt":
                    result = 1 / np.sqrt(distance)
                elif mode == "linear":
                    result = 1 / distance
                elif mode == "binary":
                    result = 1 if distance < config.gap_too_large_threshold else 0
                elif mode == "log":
                    result = 1 / np.log(distance)
                else:
                    print("Error: unknown mode")
                    break
                if result > 0.0:
                    instance_instance_proximity_matrix[id1][id2] += result
                    instance_instance_proximity_matrix[id2][id1] += result

    #TODO rest doesnt seem to work, short fix implemented:
    # create a copy of labels that only contains instances that are in the proximity matrix

    instance_instance_proximity_matrix, deletions = remove_zeros(instance_instance_proximity_matrix)
    proximity_instances = handle_deletions(instances, deletions, rows=False)
    
    return instance_instance_proximity_matrix, proximity_instances

In [21]:
instance_instance_proximity_matrix, proximity_instances = calculate_proximity_matrix(config, pos_in_paper, instances)

IndexError: index 338 is out of bounds for axis 0 with size 338

# Knowledge Graph creation

In [ ]:
# prepare csv file again
# process,software,data item,data model,data format specification,interchange format,data visualization,data validation,inference,source

@time_function
def knowledge_graph_population(config:Config, instance_types_dicts, property_types_dicts, instance_instance_proximity_matrix, proximity_instances):
    columns = list(instance_types_dicts.keys()) 
    # columns += list(property_types_dicts.keys())
    # columns = ['process', 'software', 'data item', 'data model', 'data format specification', 'data visualization', 'data validation', 'inference']



    rows = []
    for c_ID, column in enumerate(columns):
        for instance in instance_types_dicts[column]:
            # add the instance to the csv with each of their relations
            if instance not in proximity_instances:
                continue
            instance_index = proximity_instances.index(instance)
            for oc_ID, other_column in enumerate(columns):
                if other_column not in instance_types_dicts:
                    if other_column in property_types_dicts:
                        #TODO: handle properties specially
                        continue
                    continue
                if other_column != column:
                    other_column_instances = instance_types_dicts[other_column]
                    for other_instance in other_column_instances:
                        if other_instance not in proximity_instances:
                            continue
                        other_instance_index = proximity_instances.index(other_instance)
                        if instance_instance_proximity_matrix[instance_index][other_instance_index] > 0:
                            # build row column by column
                            row = [''] * len(columns)
                            row[c_ID] = instance
                            row[oc_ID] = other_instance
                            rows.append(row)

    # write to csv
    filepath = os.path.join(config.get_output_path(),'instance_relations.csv')
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(config.csv_separator.join(columns) + '\n')
        for row in rows:
            f.write(config.csv_separator.join(row) + '\n')
    
knowledge_graph_population(config, instance_types_dicts, property_types_dicts, instance_instance_proximity_matrix, proximity_instances)

In [ ]:
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

def get_rules(matrix, columns):
    # AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'
    dataframe = pd.DataFrame(matrix, columns=columns).astype(bool)

    # for each process:
    # create one res

    res = apriori(dataframe, min_support=0.4, use_colnames=True, max_len=2)

    # visualize res
    res = res.sort_values(by='support', ascending=False)
    res = res.reset_index(drop=True)
    # res

    rules = association_rules(res)
    # sort rules by confidence
    # rules = rules.sort_values(by='confidence', ascending=False)
    rules = rules.sort_values(by='lift', ascending=False) # (propably most important)
    # rules = rules.sort_values(by='leverage', ascending=False)
    # export rules to csv
    return rules

rules = get_rules(paper_instance_occurrence_matrix, instances)

In [ ]:
def identify_cross_type_rules(rules):
    cross_type = [False] * len(rules)

    for i, antecentent in enumerate(rules.antecedents):
        antecentent, = antecentent
        consequent, = rules.iloc[i].consequents
        type1, type2 = None, None
        for type in instance_types_dicts:
            if antecentent in instance_types_dicts[type]:
                type1 = type
            if consequent in instance_types_dicts[type]:
                type2 = type
            if type1 and type2:
                break
        if type1 != type2:
            cross_type[i] = True
            # print(rules.iloc[i])

    # create a copy for all rules that are cross type
    rules_cross_type = rules[cross_type].copy()
    return rules_cross_type

rules_cross_type = identify_cross_type_rules(rules)

# Output

In [ ]:
# config.visualize = True

## Functions

In [ ]:
# represent a dict
import csv
import os
from itables import init_notebook_mode, show

# better represent dataframes
if not config.for_git:
    init_notebook_mode(all_interactive=True)


def prep_dict(input_dict):
    changes_needed = {}
    # get ONLY THE FIRST key and value   
    key, value = next(iter(input_dict.items()))     
    key_change = ""
    value_change = ""
    if isinstance(key, frozenset):
        key_change = "str"
    if isinstance(value, set):
        value_change = "list"
    if isinstance(value, dict):
        value_change = prep_dict(value)
    changes_needed[key_change] = value_change
    return changes_needed
    
def change_dict(input_dict, changes_needed):
    for key, value in changes_needed.items():
        if key == "str" and value == "list":
            input_dict = {str(key): list(value) for key, value in input_dict.items()}
        elif key == "str":
            input_dict = {str(key): value for key, value in input_dict.items()}
        elif value == "list":
            input_dict = {key: list(value) for key, value in input_dict.items()}
        elif key == "str" and value == "dict":
            input_dict = {str(key): change_dict(value, changes_needed["str"]) for key, value in input_dict.items()}
        elif value == "dict":
            input_dict = {key: change_dict(value, changes_needed[""]) for key, value in input_dict.items()}
    return input_dict

def process_list(config:Config, input_list:list, filename="some_list", path=None):
    if path is None:
        path = config.get_output_path()
    filepath = os.path.join(path, filename)
    with open(filepath + '.txt', 'w', encoding="utf-8") as f:
        for item in input_list:
            f.write(f"{item}\n")

def process_dict(config:Config, input_dict:dict, filename="some_dict", path=None):
    # convert all sets to lists
    changes_needed = prep_dict(input_dict)
    processed_dict = change_dict(input_dict, changes_needed)

    if path is None:
        path = config.get_output_path()
    filepath = os.path.join(path, filename)
    with open(filepath + '.json', 'w', encoding="utf-8") as f:
        json.dump(processed_dict, f, ensure_ascii=False, indent=4)

    # TODO: Only do this for dicts that need statistical analysis
    requires_analysis = False
    value = list(processed_dict.values())[0]  # Convert dict_values to list to make it subscriptable
    if isinstance(value, dict):
        value = list(value.values())[0]  # Convert dict_values to list to make it subscriptable
        if isinstance(value, int) or isinstance(value, float):
            requires_analysis = True
    if not requires_analysis:
        return
    
    container = [
        ["Instance", "Min", "Max", "Mean", "Median", "Std"]
    ]

    for instance, papers in input_dict.items():

        # print(f"Instance: {instance}")
        gaps = papers.values()
        # generate all kinds of statistical values
        min_gap = min(gaps)
        max_gap = max(gaps)
        mean_gap = sum(gaps) / len(gaps)
        median_gap = np.median(list(gaps))
        std_gap = np.std(list(gaps))
        container.append([instance, min_gap, max_gap, mean_gap, median_gap, std_gap])

    filepath = os.path.join(path, filename)

    # TODO: Handle CSV separator
    # if not for_git:
    # Function to convert a single value
    # def convert_decimal_delimiter(value, decimal=CSV_DECIMAL):
    #     if isinstance(value, float):
    #         return f"{value}".replace('.', decimal)
    #     return value

    # # Convert all floats in your container to strings with the desired decimal delimiter
    # container = [[convert_decimal_delimiter(value) for value in row] for row in container]

    # write to csv
    with open(filepath + ".csv", 'w', newline='') as file:
        writer = csv.writer(file, delimiter=config.csv_separator)
        writer.writerows(container)   

def process_dataframe(config:Config, input_df, name = "some_df", path=None):
    if path is None:
        path = config.get_output_path()
    filepath = os.path.join(path, name)
    
    input_df.to_csv(filepath + '.csv', sep=config.csv_separator, decimal=config.csv_decimal)
    show(input_df)

In [ ]:
# visualize co-occurrences
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math

def visualize_matrix(config:Config, matrix: np.ndarray, rows: list[str], columns: list[str] = None, name: str = 'some_matrix', format = '.png', path = None) -> None:
    """
    Visualizes a matrix as a heatmap.
    matrix: The matrix to visualize
    rows: The labels for the rows
    columns: The labels for the columns
    name: The name of the file to save
    format: The format of the file to save (default: '.png', also accepts '.svg' and '.pdf', also accepts a list of formats)
    """
    if columns is None:
        columns = rows

    if path is None:
        path = config.get_output_path(visualization=True)

    ## Calculate the maximum size of the plot
    dpi = 300
    max_dpi = 600
    if config.for_git:
        dpi = 96
        max_dpi = 200
    max_pixel = 2**16  # Maximum size in any direction
    max_size = max_pixel / dpi  # Maximum size in any direction
    max_size_total = max_size * max_size # Maximum size in total
    max_size_total *= 0.05 # produce smaller files

    # Experience value of space required per cell
    factor = 0.18
    size_x: float = 2 + len(columns) * factor
    size_y: float = 3 + len(rows) * 0.8 * factor

    while size_x * size_y < max_size_total and dpi < max_dpi:
        dpi /= 0.95 
        max_size_total *= 0.95

    if dpi > max_dpi:
        dpi = max_dpi

    while size_x * size_y > max_size_total:
        dpi *= 0.95 
        max_size_total /= 0.95

    fig, ax = plt.subplots(figsize=(size_x, size_y), dpi=dpi)

    cax = ax.matshow(matrix, cmap='viridis')

    # use labels from instance_occurrences
    ax.set_xticks(range(len(columns)))
    ax.set_xticklabels(list(columns), fontsize=10, rotation=90)
    ax.set_yticks(range(len(rows)))
    ax.set_yticklabels(list(rows), fontsize=10)

    # # adjust the spacing between the labels
    # plt.gca().tick_params(axis='x', which='major', pad=15)
    # plt.gca().tick_params(axis='y', which='major', pad=15)

    # show the number of co-occurrences in each cell, if greater than 0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i, j] == 0:
                continue
            # if co_occurrences[i, j] > 100:
            #     continue
            
            # make sure the text is at most 3 digits and a dot
            decimals = 2
            if matrix[i, j] > 99:
                decimals = 0
            elif matrix[i, j] > 9:
                decimals = 1
            cell_text = round(matrix[i, j], decimals)
            if decimals == 0:
                cell_text = int(cell_text)
            plt.text(j, i, cell_text, ha='center', va='center', color='white', fontsize=4)


    # plt.show()
    fig.tight_layout()

    # title
    plt.title(name)

    if isinstance(format, list):
        for f in format:
            if f[0] != '.':
                f = '.' + f
            filepath = os.path.join(path, name + f)
            fig.savefig(filepath)
    else:
        if format[0] != '.':
            format = '.' + format
        filepath = os.path.join(path, name + format)
        fig.savefig(filepath)

def visualize_matrix_graph(config:Config, matrix, instances, instance_types_dicts, name='some_matrix_graph', path=None, node_size_mode = "sqrt", raise_mode = "prune"):
    if path is None:
        path = config.get_output_path(path, visualization=True)

    SEED = 17
    K_SPRRING = 18
    MIN_VALUE = 0.01

    scale = len(instances) * .12
    # Create a new figure
    x = scale / 10 * 16
    y = scale / 10 * 9
    fig = plt.figure(figsize=(x, y))

    # normalize the proximity matrix
    matrix = matrix / matrix.max()

    
    # Make sure the matrix is not completely stretched out
    if matrix.min() < MIN_VALUE:
        if raise_mode == "prune":
            # remove every value that is below MIN_VALUE
            matrix = np.where(matrix < MIN_VALUE, 0, matrix)
        elif raise_mode == "sqrt":
            while np.min(matrix[np.nonzero(matrix)]) < MIN_VALUE:
                matrix = np.sqrt(matrix)
        else:
            raise ValueError("Unknown raise mode")

    # alternatives are:
    # "linear" - take proximity as is
    # "sqrt" - sqrt(proximity)
    # "log" - log(proximity)
    if node_size_mode == "log":
        # TODO: see how this works with log(1)
        nodesize_map = [np.log(matrix[:, i].sum() + 1) for i in range(len(instances))]
    elif node_size_mode == "sqrt":
        nodesize_map = [np.sqrt(matrix[:, i].sum()) for i in range(len(instances))]
    elif node_size_mode == "linear":
        nodesize_map = [matrix[:, i].sum()for i in range(len(instances))]
    else:
        nodesize_map = [matrix[:, i].sum() for i in range(len(instances))]
        
    # print(max(nodesize_map))
    # print(min(nodesize_map))

    nodesize_map = np.array(nodesize_map) / max(nodesize_map) * 1000

    # print(max(nodesize_map))
    # print(min(nodesize_map))


    # Create a graph from the proximity matrix
    G = nx.from_numpy_array(matrix)

    # Specify the layout
    pos = nx.spring_layout(G, seed=SEED, k=K_SPRRING/math.sqrt(G.order()))  # Seed for reproducibility

    color_map = []

    color = {
        "process": "#1f77b4",  # muted blue
        "software": "#ff7f0e",  # safety orange
        "data item": "#2ca02c",  # cooked asparagus green
        "data model": "#d62728",  # brick red
        "data format specification": "#9467bd",  # muted purple
        "interchange format": "#8c564b",  # chestnut brown
        # "source": "#e377c2",  # raspberry yogurt pink
    }

    for instance in instances:
        added = False
        for instance_type in instance_types_dicts:
            if instance in instance_types_dicts[instance_type]:
                color_map.append(color[instance_type])
                added = True
                break
        if not added:
            color_map.append("grey")

    # Draw the graph
    options = {
        "edge_color": "grey",
        "linewidths": 0.5,
        "width": 0.5,
        "with_labels": True,  # This will add labels to the nodes
        "labels": {i: label for i, label in enumerate(instances)},
        "node_color": color_map,
        "node_size": nodesize_map,
        # "edge_color": "white",
        # "alpha": 0.9,
    }

    # print(nx.is_weighted(G))


    # nx.set_edge_attributes(G, values = 1, name = 'weight')

    nx.draw(G, pos, **options, ax=fig.add_subplot(111))

    # Make the graph more spacious
    fig.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.9)

    # Create a patch for each color
    patches = [mpatches.Patch(color=color[key], label=key) for key in color]

    # Add the legend to the graph
    plt.legend(handles=patches, loc='upper right', fontsize='x-large')


    plt.show()

    # save plot to file
    filepath = os.path.join(path, name)
    fig.savefig(filepath + '.png')
    fig.savefig(filepath + '.svg')

    # nx.get_edge_attributes(G, 'weight')

def sankey(config:Config, matrix, instances, instance_types_dicts, name='some_sankey', path = None):
    #TODO: Implement a method to create one graph per Process
    if path is None:
        path = config.get_output_path(path, visualization=True)
    # Convert the proximity matrix into a list of source nodes, target nodes, and values
    sources = []
    targets = []
    values = []

    x_pos=[0] * len(instances)
    y_pos=[0] * len(instances)
    color_map=[0] * len(instances)

    max_types = len(instance_types_dicts)
    type_positions = [0.1 + (i / max_types) * 0.8 for i in range(max_types)]

    color = {
        "process": "#1f77b4",  # muted blue
        "software": "#ff7f0e",  # safety orange
        "data item": "#2ca02c",  # cooked asparagus green
        "data model": "#d62728",  # brick red
        "data format specification": "#9467bd",  # muted purple
        "interchange format": "#8c564b",  # chestnut brown
        # "source": "#e377c2",  # raspberry yogurt pink
    }
    color = list(color.values())

    space = {}

    for i in range(matrix.shape[0]):
        source_type = None

        for j in range(matrix.shape[1]):
            target_type = None
            
            for type_depth, type in enumerate(instance_types_dicts):
                if instances[i] in instance_types_dicts[type]:
                    source_type = type_depth
                if instances[j] in instance_types_dicts[type]:
                    target_type = type_depth

            # only keep directly forward moving connections
            if target_type - source_type != 1:
                continue

            # only keep forward moving connections
            if target_type - source_type <= 0:
                continue

            if source_type not in space:
                space[source_type] = {}
            if i not in space[source_type]:
                space[source_type][i] = 0
            space[source_type][i] += matrix[i][j]
            
            if target_type not in space:
                space[target_type] = {}
            if j not in space[target_type]:
                space[target_type][j] = 0
            space[target_type][j] += matrix[i][j]

            x_pos[i] = type_positions[source_type]
            x_pos[j] = type_positions[target_type]
            color_map[i] = color[source_type]
            color_map[j] = color[target_type]
            if matrix[i][j] > 0.0:  # Ignore zero values
                sources.append(i)
                targets.append(j)
                values.append(matrix[i][j])

    for type in space:
        sum_values = sum(space[type].values())
        space[type] = {k: v/sum_values for k, v in sorted(space[type].items(), key=lambda item: item[1], reverse=True)}

    # assign each instance a proper y position
    for type in space:
        bottom = 0.1
        for i, instance in enumerate(space[type]):
            y_pos[instance] = bottom
            bottom += space[type][instance] * 0.8

    nodes = dict(
        # pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=instances,
        color=color_map,
        x=x_pos,
        y=y_pos,
        align="right",
    )

    # Create a Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=nodes,
        link=dict(
            source=sources,
            target=targets,
            value=values
        )
    )])


    fig.update_layout(width=1920, height=1080)


    fig.update_layout(title_text="Sankey Diagram", font_size=10)
    # fig.show()
    
    filepath = os.path.join(path, name)
    fig.write_image(filepath + '.png')
    fig.write_image(filepath + '.svg')
    fig.write_html(filepath + '.html')

# Represent a matrix
def process_matrix(config:Config, matrix, rows, columns=None, name = 'some_matrix', path = None, instance_types_dicts = None, mode = "sqrt"):
    if columns is None:
        columns = rows
    if path is None:
        path = config.get_output_path()
    df = pd.DataFrame(matrix, columns=columns, index=rows)
    filepath = os.path.join(path, name)
    df.to_csv(filepath + '.csv', sep=config.csv_separator, decimal=config.csv_decimal)
    path = config.get_output_path(path, visualization=True)
    if config.visualize:
        if instance_types_dicts:
            sankey(config, matrix, rows, instance_types_dicts, name + '_sankey', path=path)
            if mode:
                visualize_matrix_graph(config, matrix, rows, instance_types_dicts, name + '_graph', path=path, node_size_mode=config.proximity_mode)
            else:
                visualize_matrix_graph(config, matrix, rows, instance_types_dicts, name + '_graph', path=path)
        visualize_matrix(config, matrix, rows, columns, name, path=path)

In [ ]:
# visualize timeline
import numpy as np
import matplotlib.pyplot as plt
import math

def visualize_timeline(config:Config, year_instance_occurrence_matrix, year_papers, instances, instance_types_dicts, name='some_timeline', path=None, recursion_depth=0, start_index=0, error_matrix=None, error_instances=None):
    if not path:
        path = config.get_output_path(path, visualization=True)
    years = list(year_papers.keys())
    max_papers = max([len(year_papers[year]) for year in years])
    yearly_papers = [len(year_papers[year]) for year in years]


    ALPHA_ERROR_LINE = 0.3
    ALPHA_ERROR_ZONE = 0.2
    ALPHA_PAPER_BAR = 0.3


    for type in instance_types_dicts:
        use = [instance in instance_types_dicts[type] for instance in instances]
        type_instances = [instance for instance, use_flag in zip(instances, use) if use_flag]
        total_occurrences = [np.sum(year_instance_occurrence_matrix[:, instances.index(instance)]) for instance in type_instances]
        type_instances_sorted = [x for _, x in sorted(zip(total_occurrences, type_instances), key=lambda pair: pair[0], reverse=True)]
        
        PARTITION_SIZE = 10
        # if error_instances is not None:
        #     PARTITION_SIZE = int(0.5 * PARTITION_SIZE)
        
        type_matrix = year_instance_occurrence_matrix[:, [instances.index(instance) for instance in type_instances_sorted]]
        factor = 1
        size_x = (2 + len(years) / 6) * factor
        size_y = (2 + max_papers / 15) * factor
        size_y_2 = (2 + PARTITION_SIZE / 2) * factor
        size_y = max(size_y, size_y_2)
        fig, ax = plt.subplots(figsize=(size_x, size_y), dpi=300)

        ax.set_xticks(range(len(years)))
        years_labels = [year if len(year_papers[year]) > 0 else '' for year in years]
        ax.set_xticklabels(years_labels, fontsize=10, rotation=90)
        
        step_size = max(1, math.ceil(max_papers / 10))
        ax.set_yticks(np.arange(0, max_papers + 1, step=step_size))
        ax.set_yticklabels([str(int(x)) for x in np.arange(0, max_papers + 1, step=step_size)], fontsize=10)
        
        # set y axis label
        ax.set_ylabel('absolute', fontsize=10)

        plt.bar(range(len(years)), yearly_papers, color='black', alpha=ALPHA_PAPER_BAR, label=f"Total papers ({sum(yearly_papers)})", zorder=0)

        line_count = 0
        i = start_index
        while line_count < PARTITION_SIZE and i < len(type_instances_sorted):
            instance = type_instances_sorted[i]
            yearly_occurrences = type_matrix[:, i]
            i_total_occurrences = yearly_occurrences.sum()
            label = f"{instance} ({i_total_occurrences})"
            values = yearly_occurrences
            line = plt.plot(range(len(years)), values, label=label, zorder=3)[0]
            line_count += 1
            if error_matrix is not None and instance in error_instances:
                color = line.get_color()
                errors = error_matrix[:, error_instances.index(instance)]
                errors_plus = yearly_occurrences + errors
                line.set_label(f"{instance} ({i_total_occurrences}-{sum(errors_plus)})")
                # Plot the error as a half transparent line on top of the normal line
                plt.plot(range(len(years)), errors_plus, color=color, alpha=ALPHA_ERROR_LINE, label=f"{instance} (w/o proximity)", zorder=2)
                line_count += 1
                # color in the area between the normal line and the error line
                plt.fill_between(range(len(years)), yearly_occurrences, errors_plus, color=color, alpha=ALPHA_ERROR_ZONE, zorder=1)
            i += 1
                
                # plt.scatter(range(len(years)), errors, color='red', label=f"{instance} (error)", zorder=1)
        stop_index = i

        plt.legend()

        plt.title(f"Number of papers covering {type} instances (#{start_index+1} to #{stop_index} of {len(type_instances_sorted)})")

        # Inset for relative values
        fig.canvas.draw()
        x_lim = ax.get_xlim()  # Get the current x-axis limits from the main plot

        bbox = ax.get_position()
        bb_left, bb_bottom = bbox.x0, bbox.y0
        bb_width, bb_height = bbox.width, bbox.height

        ax_inset = plt.axes([bb_left, 0.05, bb_width, 0.15], alpha=ALPHA_PAPER_BAR, facecolor='lightgrey')
        for i, instance in enumerate(type_instances_sorted[start_index:stop_index], start=start_index):
            yearly_occurrences = type_matrix[:, i]
            values_relative = [occurrences / papers if papers > 0 else 0 for occurrences, papers in zip(yearly_occurrences, yearly_papers)]
            line_relative = ax_inset.plot(range(len(years)), values_relative, label=f"{instance} (relative)", zorder=3)[0]

            # add the error part
            if error_matrix is not None and instance in error_instances:
                color = line_relative.get_color()
                errors = error_matrix[:, error_instances.index(instance)]
                errors_plus = yearly_occurrences + errors
                errors_relative = [error / papers if papers > 0 else 0 for error, papers in zip(errors_plus, yearly_papers)]
                if max(errors_relative) > 1:
                    print(f"Error: {instance} has a relative error > 1")
                    # throw an exception because this should never be the case:
                    # raise Exception(f"Error: relative {instance} occurence + error > 1")


                ax_inset.plot(range(len(years)), errors_relative, alpha=ALPHA_ERROR_LINE, color=color, label=f"{instance} (error, relative)", zorder=2)
                # color in the area between the normal line and the error line
                ax_inset.fill_between(range(len(years)), values_relative, errors_relative, alpha=ALPHA_ERROR_ZONE, color=color, zorder=1)
        
        ax_inset.set_xlim(x_lim)

        ax_inset.set_xticks([])
        ax_inset.set_yticks(np.arange(0, 1.1, step=0.5))
        ax_inset.set_yticklabels([f"{int(x*100)}%" for x in np.arange(0, 1.1, step=0.5)], fontsize=8)

        # set y axis label
        ax_inset.set_ylabel('relative', fontsize=10)

        plt.subplots_adjust(bottom=0.3)

        start_string = f"{start_index+1}"
        stop_string = f"{stop_index}"

        # fill up with 0 to have a constant length
        start_string = "0" * (3 - len(start_string)) + start_string
        stop_string = "0" * (3 - len(stop_string)) + stop_string

        part_appendix = f"{start_string}_to_{stop_string}"
        filepath = os.path.join(path, name)
        plt.savefig(f"{filepath}_{type.replace(' ', '_')}_{part_appendix}.png")
        plt.close()
        
        start_index = stop_index
        if start_index < len(type_instances_sorted):
            # if recursion_depth > 0:
            #     break
            visualize_timeline(config, year_instance_occurrence_matrix, year_papers, instances, {type: instance_types_dicts[type]}, name, path=path, recursion_depth=recursion_depth + 1, start_index=start_index, error_matrix=error_matrix, error_instances=error_instances)
        start_index = 0
            
if config.visualize:
    yearly_error_matrix, year_error_papers = create_year_paper_occurrence_matrix(papers_metadata, error_matrix, error_papers, is_error_matrix=True)
    visualize_timeline(config, year_instance_occurrence_matrix, year_papers, instances, instance_types_dicts, name="year_instance_occurrence_matrix", error_matrix=yearly_error_matrix, error_instances=error_instances) 

## Files

In [ ]:
process_list(config, instances, "instances")

In [ ]:
# All Dicts: instance_types_dicts, papers_metadata, instance_piece_gap
process_dict(config, instance_types_dicts, 'instance_types_dicts')
process_dict(config, papers_metadata, 'papers_metadata')

### Rules

In [ ]:
process_dataframe(config, rules, 'rules')
# rules

In [ ]:
process_dataframe(config, rules_cross_type, 'cross_type_rules')
# cross_type_rules

### Paper x Instance

In [ ]:
process_matrix(config, paper_instance_occurrence_matrix, rows=papers, columns=instances, name='paper_instance_occurrence_matrix')

In [ ]:
process_matrix(config, error_matrix, rows=error_papers, columns=error_instances, name='error_matrix')

### Instance x Instance

In [ ]:
process_matrix(config, instance_instance_co_occurrence_matrix, rows=instances, columns=instances, name='instance_instance_co_occurrence_matrix')

In [ ]:
process_matrix(config, instance_instance_proximity_matrix, rows=proximity_instances, columns=proximity_instances, name='instance_instance_proximity_matrix', instance_types_dicts=instance_types_dicts)

# Approach

## Pre-Processing
Using Completion Rating in %

### 80 %: Full Text extraction
* lacking noise removal (Headings, page numbers, ...)
* lacking line-break mending

### 100 %: Bag of Words
* The problem with BoW that the words are looked at seperatly and correlation is not really clear.


### 99 %: TF-IDF
* tf-idf only on terms

### ? %: Part Of Speech (POS) Tagging, Named Entity Recognition (NER) 
* ready, but not used currently

## Visualize

### 85 % Matrix
* CSV and Dataframe dumps work fine
* Visualization as PNG or SVG are extremely large.
  * DPI regulation works to somewhat keep this in check, but images still reach 20 MB

### 100 % Timeline
* arrange the papers on a timeline and identify the flow of:
  * Processes
  * File formats
  * software
  * ...
* Additional ideas:
  * Compare this to goolge trends

## Future Work
Using Difficulty ranked (DR) solutions:

### Step 0: Look it up

#### More visualization
* https://github.com/JasonKessler/scattertext 
* https://pypi.org/project/yellowbrick/

#### NLP Pipelines:
https://spacy.io/usage/processing-pipelines


#### BLAST: Basic Local Alignment Search Tool
  * starting point: https://academic.oup.com/bioinformatics/article/39/12/btad716/7450067

#### AMIE 3
  * https://luisgalarraga.de/docs/amie3.pdf
  * https://github.com/dig-team/amie

### Step 1: Low hanging fruits

#### 1/5 DR: multi-word detection (n-gram)
Tools:  nltk, spaCy, etc.

### Step 2: Not-to-tricky follow-up

#### 3/5 DR: Acronym Expansion
Tools: spaCy - https://spacy.io/universe/project/neuralcoref

#### 3/5 DR: CoReference resolution
Tools: spaCy - https://spacy.io/universe/project/neuralcoref or https://huggingface.co/coref/ (you can use the model out of the box)

### Step 3: Vector-magic

#### 2-4/5 DR: Word embedding
* Find out, that jpeg and png are similar

(depending on your needs) - Tools: gensim - https://www.analyticsvidhya.com/blog/2023/07/step-by-step-guide-to-word2vec-with-gensim/

#### 3/5 DR: document embedding
Tools: gensim - https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

I would also check graph embeddings, sentence embeddings, and recently there is LLM2Vec

### Step 3.1: Reaping the vector-rewards

#### 1/5 DR: clustering
Tools: sklearn

Requirements: Need to have data as numbers first. This is quite possible after generating embeddings

### Step 9: Won't be happening in this paper
* Paper classes
* Subclasses of paper classes
* model which process is a subprocess of another process